# <center>**Milestone 2**<center>

## **Import**

In [1]:
import os
import json
import gc
import pickle
import sys
import random
import pandas as pd

from tqdm import tqdm
from __future__ import annotations
from collections import defaultdict

from my_scanner import *
from my_parser import *
from my_matcher import *

[nltk_data] Downloading package stopwords to C:\Users\Feng
[nltk_data]     Wang\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Feng
[nltk_data]     Wang\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to C:\Users\Feng
[nltk_data]     Wang\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Feng
[nltk_data]     Wang\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to C:\Users\Feng
[nltk_data]     Wang\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


## **Configuration**


In [2]:
RAW_ROOT = "../30-paper"
USING_SHA256_HASH = False

# 1. Output directory
OUTPUT_DIR = "23127453"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Configuration for caching
CACHE_DIR = ".cache"
os.makedirs(CACHE_DIR, exist_ok=True)
sys.setrecursionlimit(20000)

random.seed(42)

## **Dataset rescan**


In [3]:
scan_result = scan_dataset(RAW_ROOT)

for k, v in scan_result.items():
    print(k, v["status"], v["versions"])

2211-13747 NO_TEX []
2211-13748 READY ['2211-13748v1']
2211-13749 READY ['2211-13749v1']
2211-13750 READY ['2211-13750v1', '2211-13750v2']
2211-13751 READY ['2211-13751v1']
2211-13752 READY ['2211-13752v1']
2211-13753 READY ['2211-13753v1']
2211-13754 READY ['2211-13754v1']
2211-13755 READY ['2211-13755v1', '2211-13755v2']
2211-13756 READY ['2211-13756v1', '2211-13756v2']
2211-13757 READY ['2211-13757v1', '2211-13757v2']
2211-13758 READY ['2211-13758v1']
2211-13759 READY ['2211-13759v1', '2211-13759v2']
2211-13760 READY ['2211-13760v1', '2211-13760v2']
2211-13761 READY ['2211-13761v1']
2211-13762 READY ['2211-13762v1', '2211-13762v2']
2211-13763 READY ['2211-13763v1']
2211-13764 READY ['2211-13764v1']
2211-13765 READY ['2211-13765v1']
2211-13766 READY ['2211-13766v1', '2211-13766v2', '2211-13766v3']
2211-13767 READY ['2211-13767v1']
2211-13768 READY ['2211-13768v1', '2211-13768v2']
2211-13769 READY ['2211-13769v1', '2211-13769v2']
2211-13770 READY ['2211-13770v1']
2211-13771 READY ['22

## **Version-level Multi-file Resolver**


In [4]:
pub_results = []

ready_items = [
    (pub_id, info)
    for pub_id, info in scan_result.items()
    if info["status"] == "READY"
]

for pub_id, info in tqdm(ready_items, desc="Publications"):
    for version in info["versions"]:
        version_path = f"{RAW_ROOT}/{pub_id}/tex/{version}"

        result = resolve_version(
            publication_id=pub_id,
            version_name=version,
            version_path=version_path
        )

        pub_results.append(result)

# Xem kết quả
for r in pub_results:
    print(r["publication_id"], r["version"], r["used_tex_files"])


Publications: 100%|██████████| 29/29 [00:00<00:00, 625.63it/s]

2211-13748 2211-13748v1 ['weibo.tex']
2211-13749 2211-13749v1 ['On_projections_of_tailsvA.tex']
2211-13750 2211-13750v1 ['comparingsinglettestingschemes.tex']
2211-13750 2211-13750v2 ['comparingsinglettestingschemes4.tex']
2211-13751 2211-13751v1 ['Kane_et_al_JFM_v1.tex']
2211-13752 2211-13752v1 ['main.tex', 'macros.tex', '00_abstract.tex', '01_intro.tex', '02_related.tex', '03_method.tex', '04_results.tex', '05_conclusion.tex', '07_acc.tex', '06_appendix.tex']
2211-13753 2211-13753v1 ['0-title.tex']
2211-13754 2211-13754v1 ['main.tex']
2211-13755 2211-13755v1 ['main.tex']
2211-13755 2211-13755v2 ['main.tex']
2211-13756 2211-13756v1 ['main.tex']
2211-13756 2211-13756v2 ['main.tex']
2211-13757 2211-13757v1 ['main.tex']
2211-13757 2211-13757v2 ['main.tex']
2211-13758 2211-13758v1 ['main.tex']
2211-13759 2211-13759v1 ['main.tex']
2211-13759 2211-13759v2 ['main.tex']
2211-13760 2211-13760v1 ['main.tex']
2211-13760 2211-13760v2 ['main.tex']
2211-13761 2211-13761v1 ['main.tex']
2211-13762 22

## **LaTeX Hierarchy Parser**

We parsed each LaTeX version into a hierarchical tree structure. Sectioning commands were treated as internal nodes, while only sentences, tables, and figures were considered leaf nodes, strictly following the seminar constraints. No deduplication or identifier assignment was performed at this stage.


In [5]:
def save_tree_to_cache(pub_id: str, version: str, root: FileNode) -> str:
    """
    Serialize tree to disk and return cache path.
    Returns the cache file path for later retrieval.
    """
    cache_path = os.path.join(CACHE_DIR, f"{pub_id}_{version}.pkl")
    with open(cache_path, "wb") as f:
        pickle.dump(root, f, protocol=4)
    return cache_path

def load_tree_from_cache(cache_path: str) -> FileNode:
    """Load tree from cache file."""
    with open(cache_path, "rb") as f:
        return pickle.load(f)

In [6]:
parsed_versions = []


for version_info in tqdm(
    pub_results,
    desc="Parsing LaTeX versions",
    unit="version(s)"
):
    if version_info["status"] != "RESOLVED":
        continue

    version_path = (
        f"{RAW_ROOT}/"
        f"{version_info['publication_id']}/tex/"
        f"{version_info['version']}"
    )

    try:
        root_node = parse_tex_files(
            version_path=version_path,
            tex_files=version_info["used_tex_files"]
        )

        # Save to disk instead of keeping in memory
        cache_path = save_tree_to_cache(
            pub_id=version_info["publication_id"],
            version=version_info["version"],
            root=root_node
        )

        parsed_versions.append({
            "publication_id": version_info["publication_id"],
            "version": version_info["version"],
            "cache_path": cache_path,  # Store path, not tree
            "root": None  # Placeholder
        })

        # Explicitly free memory
        del root_node
        gc.collect()

    except Exception as e:
        print(
            f"[ERROR] {version_info['publication_id']} "
            f"{version_info['version']}: {e}"
        )

Parsing LaTeX versions: 100%|██████████| 42/42 [00:02<00:00, 15.29version(s)/s]


## **Deduplication & ID Assignment**

In [7]:
"""
STEP 3 - Deduplication & Merge trees per publication

Input:
    parsed_versions: list of {
        publication_id,
        version,
        root (hierarchy tree)
    }

Output:
    final_trees: dict[publication_id] -> merged root tree
"""

# Group trees by publication_id
pub_groups = defaultdict(list)
for item in parsed_versions:
    pub_groups[item["publication_id"]].append(item)

final_trees = {}

for pub_id, versions in tqdm(pub_groups.items(), desc="Deduplicating", unit="publication(s)"):
    if not versions:
        continue
        
    # Sort versions
    versions.sort(key=lambda x: int(x["version"].split("v")[-1]) if "v" in x["version"] else 0)

    # Load Base Tree
    base_info = versions[0]
    with open(base_info["cache_path"], "rb") as f:
        base_root = pickle.load(f)

    # Normalize Base
    fast_normalize_and_id(base_root, pub_id, base_info["version"])
    
    # Build Index
    content_index = build_content_index(base_root, USING_SHA256_HASH)

    # Merge subsequent versions
    for v_info in versions[1:]:
        with open(v_info["cache_path"], "rb") as f:
            root = pickle.load(f)

        # Normalize Source
        fast_normalize_and_id(root, pub_id, v_info["version"])
        
        # Deduplicate
        deduplicate_tree(root, content_index, USING_SHA256_HASH)
        
        # Explicit cleanup
        del root
    
    final_trees[pub_id] = base_root
    
    # Periodic GC to prevent memory fragmentation on large datasets
    if len(final_trees) % 50 == 0:
        gc.collect()

print(f"DONE: merged {len(final_trees)} publications")

Deduplicating: 100%|██████████| 29/29 [00:00<00:00, 1049.63publication(s)/s]

DONE: merged 29 publications


In [8]:
# Kiểm tra nhanh
sample_pub = next(iter(final_trees))
root = final_trees[sample_pub]

print("Sample publication:", sample_pub)
print("Root node type:", root.node_type)
print("Number of children:", len(root.children))

Sample publication: 2211-13748
Root node type: document
Number of children: 16


## **Export to JSON**

In [9]:
"""
Export Structured Publication JSON

Input:
    final_trees: dict[publication_id] -> root_node

Output:
    Student ID/
        <pub_id>/
            <pub_id>.json
            metadata.json
            references.json
        ...
"""

# Serialization helper
def serialize_node(node):
    """Recursively serialize a node to dictionary."""
    return {
        "id": node.id,
        "type": node.node_type,
        "full_text": getattr(node, "full_text", ""),
        "children": [serialize_node(child) for child in node.children]
    }

# Export statistics
export_count = 0
missing_metadata = []
missing_references = []

for pub_id, root in tqdm(final_trees.items(), desc="Exporting JSON", unit="pub"):
    
    # Create publication subdirectory
    pub_output_dir = os.path.join(OUTPUT_DIR, pub_id)
    os.makedirs(pub_output_dir, exist_ok=True)
    
    # ===== 1. Export Content Tree (Parsed Hierarchy) =====
    content_json = {
        "publication_id": pub_id,
        "content_tree": serialize_node(root)
    }
    
    content_path = os.path.join(pub_output_dir, f"{pub_id}.json")
    with open(content_path, "w", encoding="utf-8") as f:
        json.dump(content_json, f, ensure_ascii=False, indent=2)
    
    # ===== 2. Copy Metadata (Original from Semantic Scholar) =====
    raw_metadata_path = os.path.join(RAW_ROOT, pub_id, "metadata.json")
    metadata_path = os.path.join(pub_output_dir, "metadata.json")
    
    if os.path.exists(raw_metadata_path):
        with open(raw_metadata_path, "r", encoding="utf-8") as f:
            metadata = json.load(f)
        with open(metadata_path, "w", encoding="utf-8") as f:
            json.dump(metadata, f, ensure_ascii=False, indent=2)
    else:
        missing_metadata.append(pub_id)
    
    # ===== 3. Copy References (Original from Semantic Scholar) =====
    raw_references_path = os.path.join(RAW_ROOT, pub_id, "references.json")
    references_path = os.path.join(pub_output_dir, "references.json")
    
    if os.path.exists(raw_references_path):
        with open(raw_references_path, "r", encoding="utf-8") as f:
            references = json.load(f)
        with open(references_path, "w", encoding="utf-8") as f:
            json.dump(references, f, ensure_ascii=False, indent=2)
    else:
        missing_references.append(pub_id)
    
    export_count += 1

print(f"Exported {export_count} publications to: {OUTPUT_DIR}")

if missing_metadata:
    print(f"\nWARNING: {len(missing_metadata)} publication(s) missing metadata.json")
    print(f"   {', '.join(missing_metadata[:5])}" + (" ..." if len(missing_metadata) > 5 else ""))

if missing_references:
    print(f"\nWARNING: {len(missing_references)} publication(s) missing references.json")
    print(f"   {', '.join(missing_references[:5])}" + (" ..." if len(missing_references) > 5 else ""))

Exporting JSON: 100%|██████████| 29/29 [00:00<00:00, 203.65pub/s]

Exported 29 publications to: 23127453


## **Extract references**

In [10]:
raw_references = {}
extract_errors = 0

for version_info in tqdm(pub_results, desc="Extracting references", unit="version"):
    
    if version_info.get("status") == "NO_TEX":
        continue

    pub_id = version_info["publication_id"]
    version = version_info["version"]
    
    version_path = f"{RAW_ROOT}/{pub_id}/tex/{version}"
    
    if not os.path.exists(version_path):
        continue

    try:
        all_files_in_dir = os.listdir(version_path)
        
        target_files = [
            f for f in all_files_in_dir 
            if f.lower().endswith(('.bib', '.bbl', '.tex'))
        ]
        
        references = extract_references_from_tex_files(
            version_path=version_path,
            tex_files=target_files 
        )
        
        if references:
            if pub_id not in raw_references:
                raw_references[pub_id] = []
            raw_references[pub_id].extend(references)
            
    except Exception as e:
        extract_errors += 1
        pass

# Deduplicate & Report
deduplicated_references = {
    pub_id: deduplicate_references(refs)
    for pub_id, refs in raw_references.items()
}

count_pubs = len(deduplicated_references)
total_refs = sum(len(refs) for refs in deduplicated_references.values())

print(f"\nPublications found: {count_pubs}/{len(pub_results)}")
print(f"Total unique references: {total_refs}")

Extracting references:   0%|          | 0/42 [00:00<?, ?version/s]

[INFO] Parsing .bib file: ../30-paper/2211-13748/tex/2211-13748v1\mybibliography.bib
[INFO] Parsing .bib file: ../30-paper/2211-13750/tex/2211-13750v1\comparingsinglettestingschemes.bib
[INFO] Parsing .bib file: ../30-paper/2211-13750/tex/2211-13750v2\comparingsinglettestingschemes4.bib
[INFO] Parsing .bib file: ../30-paper/2211-13751/tex/2211-13751v1\bibliography_v3.bib
[INFO] Parsing .bib file: ../30-paper/2211-13752/tex/2211-13752v1\egbib.bib
[INFO] Parsing .bib file: ../30-paper/2211-13754/tex/2211-13754v1\refs.bib
[WARN] Bibliography file not found: ../30-paper/2211-13755/tex/2211-13755v1\egbib.bib
[INFO] Parsing .bib file: ../30-paper/2211-13755/tex/2211-13755v2\egbib.bib
[INFO] Parsing .bib file: ../30-paper/2211-13757/tex/2211-13757v1\bib.bib
[INFO] Parsing .bib file: ../30-paper/2211-13757/tex/2211-13757v2\bib.bib
[WARN] Bibliography file not found: ../30-paper/2211-13758/tex/2211-13758v1\example.bib
[INFO] Parsing .bib file: ../30-paper/2211-13758/tex/2211-13758v1\bibliograph

Extracting references: 100%|██████████| 42/42 [00:00<00:00, 289.59version/s]

[INFO] Parsing .bib file: ../30-paper/2211-13759/tex/2211-13759v2\thisbibliography.bib
[INFO] Parsing .bib file: ../30-paper/2211-13760/tex/2211-13760v1\references.bib
[INFO] Parsing .bib file: ../30-paper/2211-13760/tex/2211-13760v2\references.bib
[INFO] Parsing .bib file: ../30-paper/2211-13761/tex/2211-13761v1\references.bib
[INFO] Parsing .bib file: ../30-paper/2211-13762/tex/2211-13762v1\egbib.bib
[INFO] Parsing .bib file: ../30-paper/2211-13762/tex/2211-13762v2\egbib.bib
[INFO] Parsing .bib file: ../30-paper/2211-13764/tex/2211-13764v1\biblio.bib
[WARN] Bibliography file not found: ../30-paper/2211-13765/tex/2211-13765v1\references.bib
[INFO] Parsing .bib file: ../30-paper/2211-13766/tex/2211-13766v1\apssamp.bib
[INFO] Parsing .bib file: ../30-paper/2211-13766/tex/2211-13766v2\apssamp.bib
[INFO] Parsing .bib file: ../30-paper/2211-13766/tex/2211-13766v3\apssamp.bib
[INFO] Parsing .bib file: ../30-paper/2211-13767/tex/2211-13767v1\references.bib
[INFO] Parsing .bib file: ../30-pap

## **Reference Cleaning**

Clean and normalize BibTeX and arXiv references for matching.

In [11]:
cleaned_bibtex = {}

for pub_id in tqdm(deduplicated_references.keys(), desc="Cleaning BibTeX", unit="pub"):
    cleaned_entries = []
    for ref in deduplicated_references[pub_id]:
        try:
            cleaned_ref = clean_bibtex_entry(ref)
            cleaned_ref['ref_id'] = ref.get('ref_id', '')
            cleaned_ref['key'] = ref.get('key', '')
            
            cleaned_entries.append(cleaned_ref)
        except Exception as e:
            continue
            
    cleaned_bibtex[pub_id] = cleaned_entries

cleaned_arxiv = {}

for pub_id in tqdm(cleaned_bibtex.keys(), desc="Loading & Cleaning arXiv Truth", unit="pub"):
    ref_json_path = os.path.join(RAW_ROOT, pub_id, "references.json")
    
    if os.path.exists(ref_json_path):
        try:
            with open(ref_json_path, 'r', encoding='utf-8') as f:
                arxiv_raw = json.load(f)
            
            cleaned_entries = []
            for arxiv_id, meta in arxiv_raw.items():
                meta['arxiv_id'] = arxiv_id 
                cleaned_ref = clean_arxiv_reference(meta)
                cleaned_entries.append(cleaned_ref)
            
            cleaned_arxiv[pub_id] = cleaned_entries
        except Exception as e:
            print(f"[WARN] Error loading references.json for {pub_id}: {e}")
            cleaned_arxiv[pub_id] = []
    else:
        cleaned_arxiv[pub_id] = []

# Summary
total_bib = sum(len(x) for x in cleaned_bibtex.values())
total_arxiv = sum(len(x) for x in cleaned_arxiv.values())
print(f"\nDONE: {total_bib} cleaned BibTeX entries vs {total_arxiv} cleaned arXiv entries.")

Loading & Cleaning arXiv Truth: 100%|██████████| 25/25 [00:00<00:00, 141.32pub/s]


DONE: 2018 cleaned BibTeX entries vs 684 cleaned arXiv entries.


## **Labeling & Dataset Construction**

Generate labeled dataset using heuristic matching and manual ground truth.

### **Manual Labeling**

In [12]:
# MANUAL PAIRS
FIXED_MANUAL_DATA = {
    "2211-13768": {
        "2008MNRAS.391.1685S": "0809-0898",
        "2014MNRAS.441.3359D": "1402-7073",
        "2021MNRAS.503..920C": "2007-02958",
        "2022arXiv220405981K": "2204-05981",
        "Agrawal11611004611": "1610-04611"
    },
    "2211-13757": {
        "3DiM": "2210-04628",
        "acronym": "2011-09584",
        "attention": "1706-03762",
        "autosdf": "2203-09516",
        "cascaded-point-completion": "2004-03327"
    },
    "2211-13767": {
        "Bapat2018": "1812-02746",
        "Bittel_2021": "2101-07267",
        "Brady2021": "2107-01218",
        "Crosson_2021": "2008-09913",
        "Farhi2016": "1602-07674"
    },
    "2211-13755": {
        "AANet": "2004-09548",
        "ACVNet": "2203-02146",
        "AcfNet": "1909-03751",
        "AnyNet": "1810-11408",
        "BI3D": "2005-07274"
    },
    "2211-13766": {
        "BittencourtDamping2022": "2301-11920",
        "Marius_Schrodinger_2022": "2211-00449",
        "asjad2022magnon": "2203-10767",
        "bourcin2022strong": "2209-14643",
        "chan2011laser": "1106-3614"
    }
}

manual_pairs = []
for pub_id, labels in FIXED_MANUAL_DATA.items():
    arxiv_pool = cleaned_arxiv.get(pub_id, [])
    bib_pool = cleaned_bibtex.get(pub_id, [])
    
    if not arxiv_pool or not bib_pool: continue
    
    for bib_key, target_id in labels.items():
        bib_entry = next((e for e in bib_pool if e.get('key') == bib_key), None)
        target_arxiv = next((e for e in arxiv_pool if e.get('arxiv_id') == target_id), None)
        
        if bib_entry and target_arxiv:
            # Base record structure
            base = {
                'pub_id': pub_id,
                'bib_key': bib_key,
                'bib_ref_id': bib_entry.get('ref_id'), 
                'bib_title_clean': bib_entry.get('normalized_title'),
                'bib_authors_clean': ", ".join(bib_entry.get('normalized_authors', [])),
                'bib_author_tokens': str(bib_entry.get('author_tokens', [])),
                'bib_year': bib_entry.get('normalized_year'),
                'source': 'manual'
            }
            
            # Positive Pair
            pos = base.copy()
            pos.update({
                'candidate_arxiv_id': target_id,
                'candidate_title_clean': target_arxiv.get('normalized_title'),
                'candidate_authors_clean': ", ".join(target_arxiv.get('normalized_authors', [])),
                'candidate_author_tokens': str(target_arxiv.get('author_tokens', [])),
                'candidate_year': target_arxiv.get('normalized_year'),
                'pair_type': 'positive_manual', 'label': 1
            })
            manual_pairs.append(pos)
            
            # Negative Pairs (Exhaustive)
            for neg in [a for a in arxiv_pool if a['arxiv_id'] != target_id]:
                neg_row = base.copy()
                neg_row.update({
                    'candidate_arxiv_id': neg['arxiv_id'],
                    'candidate_title_clean': neg.get('normalized_title'),
                    'candidate_authors_clean': ", ".join(neg.get('normalized_authors', [])),
                    'candidate_author_tokens': str(neg.get('author_tokens', [])),
                    'candidate_year': neg.get('normalized_year'),
                    'pair_type': 'negative_manual', 'label': 0
                })
                manual_pairs.append(neg_row)

print(f"Generated {len(manual_pairs)} manual pairs.")

Generated 1147 manual pairs.


### **Automatic Labeling**

In [13]:
auto_pub_ids = [p for p in cleaned_bibtex.keys() if p not in FIXED_MANUAL_DATA]
total_refs = sum(len(cleaned_bibtex[p]) for p in auto_pub_ids)
target_quota = int(total_refs * 0.10)
print(f"Target Quota (10%): {target_quota} pairs")

# 2. Collect candidates
all_potential_matches = []
for pub_id in tqdm(auto_pub_ids, desc="Scanning candidates"):
    arxiv_pool = cleaned_arxiv.get(pub_id, [])
    bib_pool = cleaned_bibtex.get(pub_id, [])
    if not arxiv_pool or not bib_pool: continue
    
    for bib in bib_pool:
        # Match
        match = find_best_match(bib, arxiv_pool, threshold=0.0)
        
        if match:
            best_id, score, _ = match
            
            if score > 0.4:
                best_arxiv = next(a for a in arxiv_pool if a['arxiv_id'] == best_id)
                all_potential_matches.append({
                    'pub_id': pub_id, 'bib': bib, 'best_arxiv': best_arxiv,
                    'score': score, 'pool': arxiv_pool
                })

# 3. Sort & Select Top K
all_potential_matches.sort(key=lambda x: x['score'], reverse=True)
selected_matches = all_potential_matches[:target_quota]

print(f"\nCandidates > 0.4: {len(all_potential_matches)}")
print(f"Selected Top {len(selected_matches)} (Min Score: {selected_matches[-1]['score']:.3f})")

# 4. Generate Auto Pairs
automatic_pairs = []
for item in tqdm(selected_matches, desc="Building Auto Dataset"):
    bib = item['bib']
    best_arxiv = item['best_arxiv']
    
    base = {
        'pub_id': item['pub_id'],
        'bib_key': bib.get('key'),
        'bib_ref_id': bib.get('ref_id'), 
        'bib_title_clean': bib.get('normalized_title'),
        'bib_authors_clean': ", ".join(bib.get('normalized_authors', [])),
        'bib_author_tokens': str(bib.get('author_tokens', [])), 
        'bib_year': bib.get('normalized_year'),
        'source': 'automatic'
    }
    
    # Positive
    pos = base.copy()
    pos.update({
        'candidate_arxiv_id': best_arxiv['arxiv_id'],
        'candidate_title_clean': best_arxiv.get('normalized_title'),
        'candidate_authors_clean': ", ".join(best_arxiv.get('normalized_authors', [])),
        'candidate_author_tokens': str(best_arxiv.get('author_tokens', [])),
        'candidate_year': best_arxiv.get('normalized_year'),
        'pair_type': 'positive_auto', 'label': 1
    })
    automatic_pairs.append(pos)
    
    # Negatives
    for neg in [a for a in item['pool'] if a['arxiv_id'] != best_arxiv['arxiv_id']]:
        neg_row = base.copy()
        neg_row.update({
            'candidate_arxiv_id': neg['arxiv_id'],
            'candidate_title_clean': neg.get('normalized_title'),
            'candidate_authors_clean': ", ".join(neg.get('normalized_authors', [])),
            'candidate_author_tokens': str(neg.get('author_tokens', [])),
            'candidate_year': neg.get('normalized_year'),
            'pair_type': 'negative_auto', 'label': 0
        })
        automatic_pairs.append(neg_row)

print(f"\nGenerated {len(automatic_pairs)} automatic pairs.")

Target Quota (10%): 156 pairs


Scanning candidates: 100%|██████████| 20/20 [00:09<00:00,  2.01it/s]



Candidates > 0.4: 552
Selected Top 156 (Min Score: 0.831)


Building Auto Dataset: 100%|██████████| 156/156 [00:00<00:00, 11676.21it/s]


Generated 4641 automatic pairs.


## **Export CSV**

Export labeled dataset for modeling.

In [14]:
final_df = pd.DataFrame(manual_pairs + automatic_pairs)

cols = [
    'pub_id', 'bib_key', 'bib_ref_id', 
    'bib_title_clean', 'bib_authors_clean', 'bib_author_tokens', 'bib_year',
    'candidate_arxiv_id', 
    'candidate_title_clean', 'candidate_authors_clean', 'candidate_author_tokens', 'candidate_year',
    'source', 'pair_type', 'label'
]

for c in cols: 
    if c not in final_df.columns: final_df[c] = None

labeled_df = final_df[cols]

print(f"Positives: {len(labeled_df[labeled_df['label']==1])}")
print(f"Negatives: {len(labeled_df[labeled_df['label']==0])}")
print(f"Total pairs: {len(labeled_df)}")

csv_output_path = "../src/labeled_dataset.csv"
labeled_df.to_csv(csv_output_path, index=False, encoding='utf-8')

print(f"Saved labeled dataset to: {csv_output_path}")

Positives: 177
Negatives: 5611
Total pairs: 5788
Saved labeled dataset to: ../src/labeled_dataset.csv
